In [32]:
import requests
import bs4
import time
import random
import sys
import pandas as pd
from datetime import date
from github import Github
import json

In [6]:
HEADERS = {
    'Connection': 'keep-alive',
    'sec-ch-ua': '" Not A;Brand";v="99", "Chromium";v="99", "Google Chrome";v="99"',
    'sec-ch-ua-mobile': '?0',
    'sec-ch-ua-platform': '"Windows"',
    'Upgrade-Insecure-Requests': '1',
    'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/99.0.4844.84 Safari/537.36',
    'Accept': 'text/html,application/xhtml+xml,application/xml;q=0.9,image/avif,image/webp,image/apng,*/*;q=0.8,application/signed-exchange;v=b3;q=0.9',
    'Sec-Fetch-Site': 'none',
    'Sec-Fetch-Mode': 'navigate',
    'Sec-Fetch-User': '?1',
    'Sec-Fetch-Dest': 'document',
    'Accept-Language': 'pl-PL,pl;q=0.9,en-US;q=0.8,en;q=0.7,zh-CN;q=0.6,zh;q=0.5,de;q=0.4,cs;q=0.3,ja;q=0.2'
    }

URL = "https://www.bezrealitky.com/listings/offer-rent/flat/prague?page={page_no}"

In [30]:
url2 = 'https://www.bezrealitky.com/properties-flats-houses/552817-nabidka-pronajem-bytu'
HEADERS =   {
                        'Connection': 'keep-alive',
                        'sec-ch-ua': '" Not A;Brand";v="99", "Chromium";v="99", "Google Chrome";v="99"',
                        'sec-ch-ua-mobile': '?0',
                        'sec-ch-ua-platform': '"Windows"',
                        'Upgrade-Insecure-Requests': '1',
                        'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/99.0.4844.84 Safari/537.36',
                        'Accept': 'text/html,application/xhtml+xml,application/xml;q=0.9,image/avif,image/webp,image/apng,*/*;q=0.8,application/signed-exchange;v=b3;q=0.9',
                        'Sec-Fetch-Site': 'none',
                        'Sec-Fetch-Mode': 'navigate',
                        'Sec-Fetch-User': '?1',
                        'Sec-Fetch-Dest': 'document',
                        'Accept-Language': 'pl-PL,pl;q=0.9,en-US;q=0.8,en;q=0.7,zh-CN;q=0.6,zh;q=0.5,de;q=0.4,cs;q=0.3,ja;q=0.2'
                        }
response2 = requests.get(url2,headers=HEADERS)
soup2 = bs4.BeautifulSoup(response2.text,"lxml")

In [79]:
x = json.loads(soup2.find("script",{"id" : "__NEXT_DATA__"}).contents[0])["props"]["pageProps"]["advert"]["poiData"]

In [51]:
json_obj_list = ['public_transport','post','shop','bank','restaurant','pharmacy','school','kindergarten','sports_field','playground']

In [69]:
json.loads(x)['public_transport']['properties']['walkDistance']

210.74

In [80]:
aaa = json.loads(x)

In [81]:
len(aaa)

10

In [71]:
for key in aaa:
    print(key)

public_transport
post
shop
bank
restaurant
pharmacy
school
kindergarten
sports_field
playground


In [75]:
poi_keys = [key for key in aaa]

In [78]:
[int(aaa[key]['properties']['walkDistance']) for key in poi_keys]

[210, 730, 374, 563, 346, 618, 388, 208, 659, 54]

In [68]:
for obj in json_obj_list:
    json.loads(x)[obj]['properties']['walkDistance']
    


210
730
374
563
346
618
388
208
659
54


In [83]:
class Page:
    def __init__(self,url):
        time.sleep(random.randrange(1,4))
        self.url = url
        self.response = requests.get(self.url,headers=HEADERS)
        self.soup = bs4.BeautifulSoup(self.response.text,"lxml")
    
    def get_html(self):
        return str(self.soup)

class ListingPage(Page):
    def get_max_pages(self):
        self.max_pages = int(self.soup.find_all("a",class_='page-link')[-2].text)
        return self.max_pages

    def get_listing_urls(self):
        self.listings_in_page = self.soup.find_all("article",class_='PropertyCard_propertyCard__qPQRK propertyCard PropertyCard_propertyCard--landscape__7grmL')
        self.apartment_urls = [listing.find("a",href=True).get("href") for listing in self.listings_in_page]
        return self.apartment_urls     

class Apartment(Page):
    def get_parameters(self):
        self.price_table = self.soup.find('table',{"class": "PriceTable_priceTable__voQsR priceTable"})
        self.price_parameters = [instance.find("th").text.replace('Basic rent','Price').replace('Utility fees','Fees') for instance in self.price_table.find_all("tr")]
        self.price_values = [instance.find("strong").text.replace('\xa0',' ') for instance in self.price_table.find_all("tr")]

        self.characteristics_table = self.soup.find('section',{"class" : "box Section_section___TusU section mb-5 mb-lg-10"})
        self.characteristics_parameters = [instance.find("th").text for instance in self.characteristics_table.find_all("tr")]
        self.characteristics_values = [instance.find("td").text for instance in self.characteristics_table.find_all("tr")]

        self.additional_information = self.soup.find_all("div",{"class" : "ParamsTable_paramsTable__Eqwx_ paramsTable"})[1]
        self.additional_parameters = [instance.text for instance in self.additional_information.find_all("td") if instance.text.find('MHD') == -1]

        self.gps_json = json.loads(self.soup.find("script",{"id" : "__NEXT_DATA__"}).contents[0])

        self.poi_json = json.loads(self.gps_json["props"]["pageProps"]["advert"]["poiData"])
        self.poi_names = [key for key in self.poi_json]
        self.poi_values = [int(self.poi_json[key]['properties']['walkDistance']) for key in self.poi_names]

        self.p = self.price_parameters + self.characteristics_parameters + self.additional_parameters + self.poi_names
        self.v = self.price_values + self.characteristics_values + [1]*len(self.additional_parameters) + self.poi_values

        self.dictionary_data = dict(zip(self.p,self.v))

        self.dictionary_data['District'] = self.soup.find("span",{"class" : "PropertyAttributes_propertyAttributesItem__kscom"}).text

        self.dictionary_data['Latitude'] = self.gps_json["props"]["pageProps"]["advert"]["gps"]["lat"]
        self.dictionary_data['Longitude'] = self.gps_json["props"]["pageProps"]["advert"]["gps"]["lng"]

        return self.dictionary_data

In [91]:
current_page = 1
max_pages = 999
wait_time = 0
df = pd.DataFrame()

while current_page <= max_pages:
    while True:
        try:
            listing_page_url = URL.format(page_no=current_page)
            listing_page_ = ListingPage(url=listing_page_url)
            apartment_url_list = listing_page_.get_listing_urls()
            max_pages = listing_page_.get_max_pages()
            print(f'Parsing page {current_page} of {max_pages}')
        except:
            wait_time = wait_time+180
            print(f'Error during parsing the listing page: {listing_page_url} at {current_page} page. Waiting {int(wait_time/60)} minutes and will try again')
            if wait_time >= 1800:
                # ListingPage(url=URL.format(page_no=current_page)).get_html()
                sys.exit("Too many trials. Arborting the script.")
            else:
                time.sleep(wait_time)
            continue
        break


    for apartment_url in apartment_url_list:
        while True:
            try:
                apartment_ = Apartment(url=apartment_url)
                apartment_parameters = apartment_.get_parameters()
                df_ = pd.DataFrame([apartment_parameters])
                df = pd.concat([df,df_],axis=0,ignore_index=True)
            except:
                wait_time = wait_time+180
                print(f'Error during parsing the apartment: {apartment_url} at {current_page} page. Waiting {int(wait_time/60)
                } minutes and will try again')
                if wait_time >= 1800:
                    # Apartment(url=apartment_url).get_html()
                    # ListingPage(url=URL.format(page_no=current_page)).get_html()
                    sys.exit("Too many trials. Arborting the script.")
                else:
                    time.sleep(wait_time)
                continue
            break

    current_page = current_page + 1
        

Parsing page 1 of 65
Error during parsing the apartment: https://www.bezrealitky.com/properties-flats-houses/723061-nabidka-pronajem-bytu at 1 page. Waiting 3.0 minutes and will try again


In [9]:
df.columns

Index(['Price', 'Fees', 'Refundable security deposit', 'Listing ID', 'Layout',
       'Avaliable from', 'Floor', 'Building type', 'Age', 'Designe',
       'Floor space', 'Fully furnished', 'Condition', 'Ownership', 'EPC',
       'Heating', 'New-build', 'Enclosed balcony', 'MHD 2 minutes on foot',
       'Cellar', 'Latitude', 'Longitude', 'District', 'Reconstruction',
       'Administrative fee', 'Balcony', 'Lift', 'Parking',
       'MHD 3 minutes on foot', 'MHD 1 minute on foot', 'Terrace',
       'Front garden 1 m²', 'Plot type', 'MHD 4 minutes on foot'],
      dtype='object')

In [87]:
a1 = Apartment(url='https://www.bezrealitky.com/properties-flats-houses/552817-nabidka-pronajem-bytu')
a1.get_parameters()

{'Price': 'CZK 20,000',
 'Fees': 'CZK 5,000',
 'Refundable security deposit': 'CZK 25,000',
 'Listing ID': '552817',
 'Layout': '4+kk',
 'Avaliable from': '9/1/2022',
 'Floor': '2',
 'Building type': 'Prefab concrete',
 'Heating': 'Distance',
 'Designe': 'Standard',
 'Floor space': '77 m²',
 'Fully furnished': 'Partly',
 'Condition': 'Very good',
 'Ownership': 'Personal',
 'EPC': 'G',
 'Reconstruction': 'Exterior',
 'Enclosed balcony': 1,
 'Lift': 1,
 'Parking': 1,
 'public_transport': 210,
 'post': 730,
 'shop': 374,
 'bank': 563,
 'restaurant': 346,
 'pharmacy': 618,
 'school': 388,
 'kindergarten': 208,
 'sports_field': 659,
 'playground': 54,
 'District': 'Prague - Žižkov',
 'Latitude': 50.087438,
 'Longitude': 14.47381}